In [ ]:
!pip install pyphen
!pip install nltk

In [1]:
!python -m spacy download es_core_news_md

  Stored in directory: C:\Users\Vanda\AppData\Local\Temp\pip-ephem-wheel-cache-93d82sp9\wheels\5a\2c\3e\f54b751b98219db5864c24e25f032433969db931582f7dd23a
Successfully built es-core-news-md

    Error: Couldn't link model to 'es_core_news_md'
    Creating a symlink in spacy/data failed. Make sure you have the required
    permissions and try re-running the command as admin, or use a
    virtualenv. You can still import the model as a module and call its
    load() method, or create the symlink manually.

    C:\Users\Vanda\Anaconda3\lib\site-packages\es_core_news_md -->
    C:\Users\Vanda\Anaconda3\lib\site-packages\spacy\data\es_core_news_md


    Download successful
    Creating a shortcut link for 'en' didn't work (maybe you don't have
    admin permissions?), but you can still load the model via its full
    package name:

    nlp = spacy.load('es_core_news_md')



In [1]:
import os
import pandas as pd
import codecs
import re
import xml.etree.ElementTree as ET
import numpy as np
import shutil
from shutil import copyfile
import emoji
import re
import operator
from collections import Counter, OrderedDict, defaultdict

In [2]:
import spacy
spacy_nlp = spacy.load('es_core_news_md')
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS

In [3]:
import pyphen
PYPHEN_DIC = pyphen.Pyphen(lang='es')
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vanda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Character flooding
eg looooove, floooood
Find words that have consecutive characters appearing in them more than 2 times

In [4]:
def CharacterFlooding(tweet):
    tknzr = TweetTokenizer()
    tweet_words = tknzr.tokenize(tweet)
    floodings = 0
    regexpURL = re.compile(r'https?:\/\/.[^\s]*|www\.[^\s]*')
    for word in tweet_words:
        if not regexpURL.search(word) and len(re.findall(r'(\w)\1{2,}',word)) > 0: 
            floodings += 1
    return floodings

### Counting Capital Letters

In [5]:
def NumberOfCapitalLetters(tweet):
    now, words = NumberOfWords(tweet)
    nocl = 0
    for word in words:
        nocl += sum(1 for c in word if c.isupper())
    avg_nocl = nocl / now
    return nocl, avg_nocl

### Counting Emoticons

In [6]:
def NumberOfEmoticons(tweet):
    emojis = [i for i in tweet if i in emoji.UNICODE_EMOJI]
    return len(emojis)

### Exploring Topics

In [7]:
import pickle
SEMCAT_word_concept_dict = pickle.load(open('C:/Users/Vanda/Documents/PAN19/data/pan19-author-profiling-training-2019-01-28/kb/SEMCAT2018_word_concept_dict.p', 'rb'), encoding='utf-8')
semcor_word_concept_dict = pickle.load(open('C:/Users/Vanda/Documents/PAN19/data/pan19-author-profiling-training-2019-01-28/kb/semcor_noun_verb.supersenses.en_word_concept_dict.p', 'rb'), encoding='utf-8')
SEMCAT_concepts = set([item for sublist in SEMCAT_word_concept_dict.values() for item in sublist])
semcor_concepts = set([item for sublist in semcor_word_concept_dict.values() for item in sublist])
# merge data
wordTopicsDict = SEMCAT_word_concept_dict
for word, concept_set in semcor_word_concept_dict.items():
    wordTopicsDict[word] = wordTopicsDict[word].union(concept_set)
# words_file = open("words_to_translate.txt", "w")
# for word, topics in wordTopicsDict.items():
#     words_file.write(word+'\n')
# words_file.close()

In [8]:
es_words = open('sp_words.txt', 'r', encoding='utf-8')
en_words = list(wordTopicsDict.keys())
en_es_dict = {}
i = 0
for line in es_words:
    es_word = (line.split())[0]
    es_word = es_word.lower() # using lower case translations
    en_word = en_words[i]
    en_es_dict[en_word] = es_word
    i += 1

In [9]:
# use spanish translation of words alongside english words (some Spanish tweets might contain English words)
import copy
es_en_wordTopicsDict = copy.deepcopy(wordTopicsDict)

for word, topics in wordTopicsDict.items():
    w = word
    translation = en_es_dict[word]
    if translation not in es_en_wordTopicsDict.keys():
        es_en_wordTopicsDict[translation] = topics
    else:
        es_en_wordTopicsDict[translation].union(topics)
topics = SEMCAT_concepts.union(semcor_concepts)
topicNames = sorted(list(topics), key=lambda t: t)
print("Number of topics: ", len(topics))
print("Topics: ", topicNames)

Number of topics:  133
Topics:  ['Tops', 'act', 'adjectives_for_people', 'animal', 'april_fool', 'art', 'artifact', 'astronomy', 'attribute', 'baseball', 'bathroom', 'beach', 'big', 'biomes', 'birds', 'birthday', 'boat', 'bodies_of_water', 'body', 'buildings', 'camping', 'car', 'carnival', 'carparts', 'castle', 'cats', 'change', 'christmas', 'circus', 'clothes', 'cognition', 'colors', 'communication', 'competition', 'computer', 'constitution', 'consumption', 'contact', 'container', 'cooking', 'cooking_tools', 'country', 'creation', 'dance', 'dentist', 'desserts', 'doctor', 'dogs', 'driving', 'election', 'emotion', 'emotions', 'energy', 'event', 'fall', 'family', 'farm', 'feeling', 'fish', 'flowers', 'food', 'foodweb', 'fruit', 'furniture', 'geography', 'grammar', 'group', 'happiness', 'happy', 'house', 'housing', 'insect', 'jobs', 'kitchen', 'land_forms', 'languages', 'leaders', 'legal', 'location', 'mammal', 'many', 'math', 'measurement', 'metals', 'military', 'money', 'motion', 'moti

In [10]:
def Topics(tweet):
    topicsFrequency = {t:0 for t in topics}
    tweet = spacy_nlp(tweet)
    lemmas = [token.lemma_ for token in tweet]
    for token in tweet:
        lemma = token.lemma_
        topicSet = wordTopicsDict.get(lemma, set())
        for t in topicSet:
            topicsFrequency[t] += 1
    topicsFrequency = OrderedDict(sorted(topicsFrequency.items(), key=lambda e: e[0]))
    topicsValues = list(topicsFrequency.values())
    return topicsValues

### Counting URLs

In [11]:
def NumberOfURLs(tweet):
    tknzr = TweetTokenizer()
    tweet_words = tknzr.tokenize(tweet)
    regexpURL = re.compile(r'https?:\/\/.[^\s]*|www\.[^\s]*')
    urlNumber = 0
    for word in tweet_words:
        if regexpURL.search(word): 
            urlNumber += 1
    return urlNumber

### Counting Repeated Words
Compute:
- number of tokens repeated more (or equal) than *k* times 
- maximum number of repetition (of a single token) (>=*k*)

For example (k=3) in the tweet: *"Hairy cats like other cats that are not hairy. However, hairy dogs like cats that are not hairy."*
- the tokens repeated more (or equal) than 3 times are *hairy* and *cats*, so the number of tokens repeated more than 3 times is 2
- the token *hairy* is repeated most of the time: 4 times

In [12]:
def wordsRepeated(tweet, k=3):
    frequency = defaultdict(int)
    tweet = spacy_nlp(tweet)
    for token in tweet:
        if token.lemma_ not in spacy_stopwords:
            frequency[token.lemma_] += 1
    maxAppearance = 0
    numberOfTokensRepeated = 0
    if len(frequency.items()) > 0:
        maxAppearance = max(frequency.items(), key=operator.itemgetter(1))[1]
        numberOfTokensRepeated = sum(1 for (lemma, freq) in frequency.items() if freq >= k)
    return maxAppearance, numberOfTokensRepeated

### Number of POS tags

In [13]:
all_pos_tags = ["NO_TAG", "ADJ", "ADP", "ADV","AUX", "CONJ","CCONJ","DET",
                      "INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM",
                      "VERB","X","EOL","SPACE"]

In [14]:
def POSTags(tweet):
    tweet = spacy_nlp(tweet)
#     pos_list = []
    c = Counter()
    c.update({x:0 for x in all_pos_tags})
    pos_list = [token.pos_ for token in tweet]
    assert len(set(pos_list).difference(set(all_pos_tags))) == 0
    c.update(pos_list)
    c = OrderedDict(sorted(c.items(), key=lambda e: e[0]))
    return list(c.values())

### Number of Sentences

In [15]:
def NumberOfSentences(tweet):
    sentences = sent_tokenize(tweet)
    return len(sentences)

### Number of Words

In [16]:
def NumberOfWords(tweet):
    tknzr = TweetTokenizer()
#     words = word_tokenize(tweet)
    tweet_words = tknzr.tokenize(tweet)
    return len(tweet_words), tweet_words

### Readability Score
using Flesch–Kincaid readability tests (higher score means the tweet is more easy to read)

In [17]:
def Readability(tweet):
    totalSentences = float(NumberOfSentences(tweet))
    totalTweetWords, tweetWords = NumberOfWords(tweet)
    totalTweetWords = float(totalTweetWords)
    regexpURL = re.compile(r'https?:\/\/.[^\s]*|www\.[^\s]*')
    totalSyllables = 0.0
    for word in tweetWords:
        if not regexpURL.search(word):
            hyphenated = PYPHEN_DIC.inserted(word)
            syllables = hyphenated.count("-") + 1 - hyphenated.count("--")
            totalSyllables += syllables
#         else:
#             totalTweetWords -= 1.0
    if totalSentences > 0 and totalTweetWords > 0:
        score = 206.835 - 1.015 * (totalTweetWords/totalSentences) - 84.6 * (totalSyllables/totalTweetWords)
    else:
        print("Readability issue")
        score = 0.0
    return score

### Feature Names

In [18]:
fNames = ['nos', 'now', 'readabilityScore', 'noclPerWord', 'noe', 'nocf', 'noURL', 'maxWordAppearancePerTweet', 'nowr']
fNames.extend(all_pos_tags)
fNames.extend(topicNames)
fNames = ['avg_'+f for f in fNames]
fNames.append('maxWordAppearancePerProfile')
fNames = sorted(fNames)
print(len(fNames))
print(fNames)
fNames_dir = 'C:/Users/Vanda/Documents/PAN19/data/pan19-author-profiling-training-2019-01-28/en_features/'
if not os.path.exists(fNames_dir):
    os.makedirs(fNames_dir)
pickle.dump(fNames, open(fNames_dir+'feature_names.p', 'wb'))

164
['avg_ADJ', 'avg_ADP', 'avg_ADV', 'avg_AUX', 'avg_CCONJ', 'avg_CONJ', 'avg_DET', 'avg_EOL', 'avg_INTJ', 'avg_NOUN', 'avg_NO_TAG', 'avg_NUM', 'avg_PART', 'avg_PRON', 'avg_PROPN', 'avg_PUNCT', 'avg_SCONJ', 'avg_SPACE', 'avg_SYM', 'avg_Tops', 'avg_VERB', 'avg_X', 'avg_act', 'avg_adjectives_for_people', 'avg_animal', 'avg_april_fool', 'avg_art', 'avg_artifact', 'avg_astronomy', 'avg_attribute', 'avg_baseball', 'avg_bathroom', 'avg_beach', 'avg_big', 'avg_biomes', 'avg_birds', 'avg_birthday', 'avg_boat', 'avg_bodies_of_water', 'avg_body', 'avg_buildings', 'avg_camping', 'avg_car', 'avg_carnival', 'avg_carparts', 'avg_castle', 'avg_cats', 'avg_change', 'avg_christmas', 'avg_circus', 'avg_clothes', 'avg_cognition', 'avg_colors', 'avg_communication', 'avg_competition', 'avg_computer', 'avg_constitution', 'avg_consumption', 'avg_contact', 'avg_container', 'avg_cooking', 'avg_cooking_tools', 'avg_country', 'avg_creation', 'avg_dance', 'avg_dentist', 'avg_desserts', 'avg_doctor', 'avg_dogs', 

### Compute features for each tweet

In [19]:
def GetFeatures(tweet):
    features = {}
    features['nos'] = NumberOfSentences(tweet)
    features['now'], words = NumberOfWords(tweet)
    features['readabilityScore'] = Readability(tweet)
    nocl, features['noclPerWord'] = NumberOfCapitalLetters(tweet)
    features['noe'] = NumberOfEmoticons(tweet)
    features['nocf'] = CharacterFlooding(tweet)
    features['noURL'] = NumberOfURLs(tweet)
    features['maxWordAppearancePerTweet'], features['nowr'] = wordsRepeated(tweet)

    pos = POSTags(tweet)
    posTags = sorted(all_pos_tags)
    for i in range(len(posTags)):
        features[posTags[i]] = pos[i]
    topicValues = Topics(tweet)
    for i in range(len(topicNames)):
        features[topicNames[i]] = topicValues[i]

    return features

### Compute average of features for each Twitter profile

In [20]:
def cummulateFeatures(profileFeatures):
    denom = len(profileFeatures)
    cummulatedFeatures = defaultdict(float)
    for tweetFeature in profileFeatures:
        for featureName in tweetFeature.keys():
            cummulatedFeatures[('avg_'+featureName)] += tweetFeature[featureName]
    for k, v in cummulatedFeatures.items():
        cummulatedFeatures[k] = cummulatedFeatures[k]/denom
    cummulatedFeatures = OrderedDict(sorted(cummulatedFeatures.items(), key=lambda e: e[0]))
    featureNames = list(cummulatedFeatures.keys())
    features = list(cummulatedFeatures.values())
    return features, featureNames

In [21]:
def writeProfileFeatures(directory, profileID, profileFeatures, Truth, TweetGenerator):
    profileName = Truth[0][profileID]
    features, featureNames = cummulateFeatures(profileFeatures)
#     features, featureNames = additionalProfileFeatures(TweetGenerator,  features, featureNames)
    features = [str(f) for f in features]
    features = '\t'.join(features)
    with codecs.open(directory+'/features.txt', "a", "utf-8-sig") as text_file:
        text_file.write(profileName+'\t'+features+'\t'+Truth[1][profileID]+'\t'+Truth[2][profileID]+'\n')

### Max number of word repetitions for a profile

In [22]:
def maxWordRepetitionsPerProfile(TweetGenerator):
    tknzr = TweetTokenizer()
    frequency = defaultdict(int)
    for tweet in TweetGenerator:
        tweet = tweet['data']
        tweet = spacy_nlp(tweet)
        for token in tweet:
            if token.lemma_ not in spacy_stopwords:
                frequency[token.lemma_] += 1
    maxAppearance = 0
    if len(frequency.items()) > 0:
        maxAppearance = max(frequency.items(), key=operator.itemgetter(1))[1]
    return maxAppearance

In [23]:
def additionalProfileFeatures(TweetGenerator, features, featureNames):
    maxWordAppearance = maxWordRepetitionsPerProfile(TweetGenerator)
    features.append(maxWordAppearance)
    featureNames.append('maxWordAppearancePerProfile')
    return features, featureNames

### Load Profile & Process Tweets

In [24]:
def LoadProfile(input_file):    
    Profile = ET.parse(input_file)
    ProfileRoot = Profile.getroot()
    Profile_attr = ProfileRoot.attrib
    for tweet in Profile.iter('document'):        
        tweet_dict = Profile_attr.copy()
        tweet_dict.update(tweet.attrib)
        tweet_dict['data'] = tweet.text
        yield tweet_dict

In [ ]:
def ProcessFolder(root_directory, input_directory, output_directory):    
    #Create output directory (if it does not exist yet)
    profile_directory = root_directory+'/'+output_directory+'/'+'profile'
    tweet_directory = root_directory+'/'+output_directory+'/'+'tweet'
    if os.path.exists(root_directory+'/'+output_directory):
        shutil.rmtree(root_directory+'/'+output_directory)
    os.mkdir(root_directory+'/'+output_directory)
    os.mkdir(profile_directory)
    os.mkdir(tweet_directory)
    #Read labels (and file names)
    Truth = pd.read_csv(root_directory+'/'+input_directory+'/truth.txt', sep=":::", header=None, engine='python')
    #Iterate over all user names, and process the corresponding file names
    for i in range(0,Truth.shape[0]):
#     for i in range(0,2):        
        #Open text file for output   
        print(Truth[0][i])
        profileFeatures = []
        TweetGenerator = LoadProfile(root_directory+'/'+input_directory+'/'+Truth[0][i]+'.xml')
        with codecs.open(tweet_directory+'/'+Truth[0][i]+'.txt', "w", "utf-8-sig") as text_file: 
            for tweet in TweetGenerator:
                tweetFeatures = GetFeatures(tweet['data'])
                profileFeatures.append(tweetFeatures)
                tweetFeatures = [str(f) for f in tweetFeatures.values()]
                tweetFeatures = '\t'.join(tweetFeatures)
                text_file.write(tweetFeatures + '\n')   
        TweetGenerator = LoadProfile(root_directory+'/'+input_directory+'/'+Truth[0][i]+'.xml')
        writeProfileFeatures(profile_directory, i, profileFeatures, Truth, TweetGenerator)

In [ ]:
ProcessFolder('C:\\Users\\Vanda\\Documents\\PAN19\\data\\pan19-author-profiling-training-2019-01-28', 'es', 'es_features')

acfdc9e43ed5efa10a903453261a3c12
40adbb05f96fdd96f767b5967458faf1
4aa2fb302140ec35cc6bc8a0d7d35f6
58db587d884d22afefbcd37aa26af458
dd9494d1bff7fa477cc03fea5294a510
6ccf89c7b178b36d95d241c1c5f3da23
b8ab53c54215915da53006d415aadc35
9616ecddb159ad7ad99421903ac05499
8b6826222306fd4694afbd709a1cb52c
c75d850ee43e714116bf7c7b6a4f7793
d4f83ae6c529a0379bbf48fa02a5bbda
23fb30684841d6a5444efd32ad75e240
d9bb54f64ab9f077398c858e6ccee805
888fb276582537e6cbb354a4925bbbbf
4f8b1d79f22eaaa45b6832c80e7d1b6b
e316ea877ed9602e1df08df6a3064ce0
7f73cce03dfbcdd6af4a61249c737a7c
af2d03f136310f46566c9e744741857a
5deafa3dd19c9c88628ec36c21dd8d4d
f180c6c1d4a11b4df97610f88ba856ef
17e41f62a2a464e816391675705dbbfb
c25555f6b5c8ec8257ca11c34e6a35da
b5ac8bf7cb4513ed69810235716d7211
6d5dfde2e326bb581217e50059307f0
da17f2e8fcc0a11c6ae3963aea7c7f7d
489c96c091bff6308a80e82a85766cd9
94b820618fd3e05791e9b34ed8765a31
22623bbd527fe2bdee9a009a8539c0e8
c26253914fb0c6dbcabdf8a763a571dc
92df748ec8d5f55f3d6bc25d7a9e3125
61b76989cd71